In [ ]:
%matplotlib inline

# Exploring sophisticated learning rate strategies for deep learning with PyTorch

## Importing the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import torch
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch import optim
import torchvision
from torchvision import transforms, datasets

## Setting up the hardware backend (GPU/CPU)

In [ ]:
force_cpu = False
if torch.cuda.is_available() and not force_cpu:
    use_gpu = True
    FloatTensor = torch.cuda.FloatTensor
    LongTensor = torch.cuda.LongTensor
else:
    use_gpu = False
    FloatTensor = torch.FloatTensor
    LongTensor = torch.LongTensor

## Helper functions and classes

### Moving average

In [ ]:
def mv_avg(l, n):
    n = min(n, len(l))
    s = sum(l[-n:])
    return s / n

### Classes for keeping track of performance information

In [ ]:
class PerformanceHistory:
    
    def __init__(self, mv_avg_w=1):
        self.losses = []
        self.accs = []
        self.mv_avg_w = mv_avg_w
        self.mv_avg_losses = []
        self.mv_avg_accs = []
        
    def update_history(self, loss, acc):
        self.losses.append(loss)
        mv_avg_loss = mv_avg(self.losses, self.mv_avg_w)
        self.mv_avg_losses.append(mv_avg_loss)
        self.accs.append(acc)
        mv_avg_acc = mv_avg(self.accs, self.mv_avg_w)
        self.mv_avg_accs.append(mv_avg_acc)
        
class TrainingHistory(PerformanceHistory):
    
    def __init__(self):
        super().__init__(32)
        self.lrs = []
        
    def update_history(self, lr, loss, acc):
        super().update_history(loss, acc)
        self.lrs.append(lr)

## Importing the dataset

In [ ]:
batch_size = 32

training_set_transform = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.9, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
training_set = datasets.CIFAR10(root='CIFAR10_data', train=True,
                               transform=training_set_transform,
                               download=True)
training_set_loader = torch.utils.data.DataLoader(training_set,
                                                 batch_size=batch_size,
                                                 shuffle=True,
                                                 num_workers=4)

test_set_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_set = datasets.CIFAR10(root='CIFAR10_data', train=False,
                           transform=test_set_transform,
                           download=True)
test_set_loader = torch.utils.data.DataLoader(test_set,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             num_workers=4)

### Visualizing the data

In [ ]:
images, _ = next(iter(training_set_loader))
grid = torchvision.utils.make_grid(images, normalize=True)
grid = np.transpose(grid.numpy(), (1, 2, 0))
plt.imshow(grid)

## Creating the network architecture

In [ ]:
class ConvBnLayer(nn.Module):
    
    def __init__(self, in_channels, out_channels,
                 kernel_size, stride, padding):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

class ResLayer(ConvBnLayer):
    
    def forward(self, x):
        return x + super().forward(x)

class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = ConvBnLayer(3, 32, 5, 1, 2)
        self.layer1 = nn.Sequential(
            ConvBnLayer(32, 64, 3, 2, 1),
            ResLayer(64, 64, 3, 1, 1),
            ResLayer(64, 64, 3, 1, 1))
        self.layer2 = nn.Sequential(
            ConvBnLayer(64, 128, 3, 2, 1),
            ResLayer(128, 128, 3, 1, 1),
            ResLayer(128, 128, 3, 1, 1)
        )
        self.avgpool = nn.AdaptiveAvgPool2d(4)
        self.fc = nn.Linear(2048, 10)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.avgpool(x)
        x = self.dropout(x.view(x.size(0), -1))
        x = self.fc(x)
        return x
    
def create_model():
    model = CNN()
    if use_gpu:
        model = model.cuda()
    return model

## Implementing the learning rate schedulers

In [ ]:
class _Scheduler:
    
    def __init__(self, optimizer):
        self.i = 0
        self.optimizer = optimizer
        self.lr = 0
        self.snapshot = False
        
    def calc_lr(self):
        return 0
    
    def step(self):
        self.lr = self.calc_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.lr
        self.i += 1
        return self.snapshot

class CosineAnnealing(_Scheduler):
    
    def __init__(self, optimizer, min_lr, max_lr, cycle_len, cycle_mult):
        super().__init__(optimizer)
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.cycle_len = cycle_len
        self.i_max = self.cycle_len - 1
        self.cycle_mult = cycle_mult
        
    def calc_lr(self):
        # linearly scale iteration to be between 0 and pi
        # so cosine is between -1 and 1
        x = self.i / self.i_max * np.pi
        # take cosine of scaled iteration and linearly
        # scale it to be between min_lr and max_lr
        lr = (self.max_lr - self.min_lr) / 2 * (np.cos(x) + 1) + self.min_lr
        return lr
    
    def step(self):
        _ = super().step()
        if self.i > self.i_max:
            self.i = 0
            self.cycle_len *= self.cycle_mult
            self.i_max = self.cycle_len - 1
            self.snapshot = True
        else:
            self.snapshot = False
        return self.snapshot

class Exponential(_Scheduler):
    
    def __init__(self, optimizer, base_lr=5e-6, n=1.01):
        super().__init__(optimizer)
        self.base_lr = base_lr
        self.n = n
        
    def calc_lr(self):
        lr = self.base_lr * self.n ** self.i
        return lr

## Implementing the classification model

In [ ]:
class ImageClassifier:
    
    def __init__(self, snapshot_ensemble_size=0):
        self.model = create_model()
        self.snapshots = []
        self.snapshot_ensemble_size = snapshot_ensemble_size
        self.optimizer = optim.Adam(self.model.parameters())
        self.scheduler = None
        self.train_history = TrainingHistory()
        self.test_history = PerformanceHistory()

    def forward_pass(self, model, data):
        inputs, targets = data
        inputs = Variable(inputs.type(FloatTensor))
        targets = Variable(targets.type(LongTensor))
        outputs = model(inputs)
        _, predictions = outputs.max(1)
        loss = F.cross_entropy(outputs, targets)
        acc = (predictions == targets).sum().data[0] / batch_size
        return loss, acc
    
    def lr_find(self, epochs=1):
        lrf_model = copy.deepcopy(self.model)
        lrf_optimizer = optim.Adam(lrf_model.parameters())
        lrf_scheduler = Exponential(lrf_optimizer)
        self.lrf_history = TrainingHistory()
        for epoch in range(1, epochs+1):
            with tqdm(training_set_loader,
                      desc="[lr_find] Epoch %d/%d" % (epoch, epochs),
                      unit="batches") as t:
                for data in t:
                    lrf_optimizer.zero_grad()
                    loss, acc = self.forward_pass(lrf_model, data)
                    loss.backward()
                    _ = lrf_scheduler.step()
                    lrf_optimizer.step()
                    self.lrf_history.update_history(lrf_scheduler.lr,
                                                    loss.data[0], acc)
                    t.set_postfix(loss=self.lrf_history.mv_avg_losses[-1],
                                  acc=self.lrf_history.mv_avg_accs[-1],
                                  lr=lrf_scheduler.lr)
                    starting_loss_i = min(31, len(self.lrf_history.mv_avg_losses)-1)
                    loss_threshold = self.lrf_history.mv_avg_losses[starting_loss_i] * 1.3
                    if self.lrf_history.mv_avg_losses[-1] > loss_threshold:
                        break
        print("Done!")
                        
    def set_scheduler(self, scheduler, opts):
        self.scheduler = scheduler(self.optimizer, *opts)

    def train(self, epochs=10):
        if self.scheduler == None:
            print("Scheduler not yet set")
            return
        for epoch in range(1, epochs+1):
            with tqdm(training_set_loader,
                      desc="[train] Epoch %d/%d" % (epoch, epochs),
                      unit="batches") as t:
                for data in t:
                    self.optimizer.zero_grad()
                    loss, acc = self.forward_pass(self.model, data)
                    loss.backward()
                    snapshot = self.scheduler.step()
                    if self.snapshot_ensemble_size and snapshot:
                        self.snapshots.append(copy.deepcopy(self.model))
                        if len(self.snapshots) > self.snapshot_ensemble_size:
                            del self.snapshots[0]
                    self.optimizer.step()
                    self.train_history.update_history(self.scheduler.lr,
                                                      loss.data[0], acc)
                    t.set_postfix(loss=self.train_history.mv_avg_losses[-1],
                                  acc=self.train_history.mv_avg_accs[-1])
            running_test_loss = 0.0
            running_test_acc = 0.0
            i = 1
            if len(self.snapshots) == 0:
                test_ensemble = [self.model]
            else:
                test_ensemble = self.snapshots
            with tqdm(test_set_loader,
                      desc="[test] Epoch %d/%d" % (epoch, epochs),
                      unit="batches") as t:
                for data in t:
                    inputs, targets = data
                    inputs = Variable(inputs.type(FloatTensor))
                    targets = Variable(targets.type(LongTensor))
                    mean_outputs = 0
                    for model in test_ensemble:
                        outputs = model(inputs)
                        mean_outputs += outputs
                    mean_outputs = mean_outputs / len(test_ensemble)
                    _, predictions = mean_outputs.max(1)
                    loss = F.cross_entropy(mean_outputs, targets)
                    acc = (predictions == targets).sum().data[0] / batch_size
                    running_test_loss += loss.data[0]
                    running_test_acc += acc
                    t.set_postfix(loss=running_test_loss/i,
                                  acc=running_test_acc/i)
                    i += 1
            self.test_history.update_history(running_test_loss/len(test_set_loader),
                                             running_test_acc/len(test_set_loader))
        print("Done!")

## Finding optimal learning rate values

In [ ]:
classifier = ImageClassifier()
classifier.lr_find()

In [ ]:
plt.semilogx(classifier.lrf_history.lrs,
             classifier.lrf_history.mv_avg_losses)
plt.xlabel("Learning rate")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.semilogx(classifier.lrf_history.lrs,
             classifier.lrf_history.mv_avg_accs)
plt.xlabel("Learning rate")
plt.ylabel("Accuracy")
plt.show()

## Training with a static learning rate

In [ ]:
classifier.set_scheduler(Exponential, (3e-4, 1))
classifier.train()

### Visualizing training performance

In [ ]:
fig, axes = plt.subplots(nrows=3)
axes[0].plot(classifier.train_history.lrs, color="Blue")
axes[0].set_ylabel("Learning Rate", color="Blue")
axes[1].plot(classifier.train_history.mv_avg_losses, color="Red")
axes[1].set_ylabel("Loss", color="Red")
axes[2].plot(classifier.train_history.mv_avg_accs, color="Green")
axes[2].set_ylabel("Accuracy", color="Green")
axes[2].set_xlabel("Mini-batch")
fig.tight_layout()
plt.show()

## Training with a cosine annealing learning rate scheduler and snapshot ensembling

In [ ]:
classifier2 = ImageClassifier(4)
classifier2.set_scheduler(CosineAnnealing, (1e-5, 5e-4, len(training_set_loader)*2, 1))
classifier2.train()

### Visualizing training performance

In [ ]:
fig, axes = plt.subplots(nrows=3)
axes[0].plot(classifier2.train_history.lrs, color="Blue")
axes[0].set_ylabel("Learning Rate", color="Blue")
axes[1].plot(classifier2.train_history.mv_avg_losses, color="Red")
axes[1].set_ylabel("Loss", color="Red")
axes[2].plot(classifier2.train_history.mv_avg_accs, color="Green")
axes[2].set_ylabel("Accuracy", color="Green")
axes[2].set_xlabel("Mini-batch")
fig.tight_layout()
plt.show()

## Comparing the results

In [ ]:
fig, axes = plt.subplots(ncols=2)
axes[0].plot(np.arange(1, len(classifier.test_history.accs)+1),
             classifier.test_history.losses, 
             color="Crimson", label="Static LR")
axes[0].plot(np.arange(1, len(classifier2.test_history.accs)+1),
             classifier2.test_history.losses, 
             color="Goldenrod", label="CA w/ SE")
axes[0].legend()
axes[0].set_ylabel("Loss")
axes[1].plot(np.arange(1, len(classifier2.test_history.accs)+1),
             classifier2.test_history.accs,
             color="Darkgreen", label="Static LR")
axes[1].plot(np.arange(1, len(classifier2.test_history.accs)+1),
             classifier2.test_history.accs,
             color="Blue", label="CA w/ SE")
axes[1].set_ylabel("Accuracy")
axes[0].set_xlabel("Epoch")
axes[1].set_xlabel("Epoch")
axes[1].legend()
plt.tight_layout()
plt.show()